# Gradient Descent

## Recap
Let $f: \mathbb{R} \to \mathbb{R}$ be our **target function** to be minimized (or maximized). This can be written as
$$ x^\ast = \argmin_{x \in \mathbb{R}} f(x). $$
$x^\ast$ is called a **global minimum** of $f$.

The gradient descent (GD) is an iterative algorithm to find $x^\ast$. The vanilla GD requires the following:
- $x_0$: initialization point
- $\eta$: learning rate
- Stopping condition

The **pseudocode** for GD is as follows:
1. **Initialize**: $x \leftarrow x_0$  
2. **Repeat until stopping condition is met**:
   $$x \leftarrow x - \eta \cdot f'(x)$$
3. **Return**: $x$

**Examples of stopping conditions**
- Number of iterations
- $|f(x_n) - f(x_{n-1})| < \epsilon$ for a very small $\epsilon$
- $|x_n - x_{n-1}| < \epsilon$ for a very small $\epsilon$
- $|f'(x_n)| < \epsilon$ for a very small $\epsilon$

Let's see how it works with a quadratic function $y = ax^2 + bx + c$ with $a > 0$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Coefficients of f
a = 3 # should be positive
b = 4
c = 5

def f(x):
    return a*x**2 + b*x + c

def der_f(x):
    return 2*a*x + b

# Hyperparameters
lr = 0.1                # learning rate eta
N = 10                  # Stopping condition: 10 iterations

# Initialization
np.random.seed(87)
x = np.random.randn()   # random number from standard normal distribution

# History
x_history = np.array([x])

# GD
for _ in range(N):
    der = der_f(x)
    x = x - lr * der    # updating
    x_history = np.append(x_history, x)

print(f"Minimizer of x: {x_history[-1]:.3f}")

y_history = np.array([f(x) for x in x_history])

x_min, x_max = x_history.min(), x_history.max()
padding = (x_max - x_min)*0.3
x_points = np.linspace(x_min - padding, x_max + padding, 100)
y_points = np.array([f(x) for x in x_points])

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x_points, y_points, label=r"$f(x) = (x - 3)^2$", color="blue")
ax.plot(x_history, y_history, 'o-r', label=r"x_n")
for i, (xi, yi) in enumerate(zip(x_history, y_history)):
    ax.text(xi, yi + 0.3, str(i), ha='center', fontsize=8, color='black')
    
ax.set_title("Gradient Descent on quadratic function")
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.legend()
ax.grid(True)

plt.show()

#### Discussion: Limitation
What conditions must the function $f$ satisfy to ensure that vanilla gradient descent converges to a **global minimum**?

# Gradient Descent in ML
## Simple linear regression
**Setup**
1. Model
$$ y = b + w x + \epsilon, \quad \epsilon \sim \mathcal{N}(0,1). $$
2. Data: $\{x_i, y_i\}$ for $i = 1, \dots, n$.
3. Goal: What is the best $(b, \ w)$ that explains the data based on the model.

**Mathematical Formulation**

Want to find 
$$ \argmin_{(b, \ w) \in \mathbb{R}^2} \frac{1}{n} \sum_{i=1}^{n} (b + w x_i - y_i)^2. $$

Target function is a bivariate function of $b$ and $w$.
$$ L(b, w) = \frac{1}{n} \sum_{i=1}^{n} (b + w x_i - y_i)^2. $$
Note that $x_i$ and $y_i$ are constants (given data) in $L$. Even though $L$ is bivariate, the logic is the same. 

We update each variable by subtracting (learning rate) $\times$ (derivative).

**Partial Derivatives**
$$ \begin{gathered}
\frac{\partial L}{\partial b} (b, w) = \frac{2}{n} \sum_{i=1}^{n} (b + w x_i - y_i), \\
\frac{\partial L}{\partial w} (b, w) = \frac{2}{n} \sum_{i=1}^{n} x_i (b + w x_i - y_i).
\end{gathered} $$

**Pseudocode**
Let $(b_0, w_0)$ be the initial point and $\eta$ be the learning rate. For pre-specified stopping condition, the pseudocode of GD is as follows:
1. **Initialize**: $b \leftarrow b_0$ and $w \leftarrow w_0$  
2. **Repeat until stopping condition is met**:
   $$ b \leftarrow b - \eta \cdot \frac{\partial L}{\partial b} (b, w)$$
   $$ w \leftarrow w - \eta \cdot \frac{\partial L}{\partial w} (b, w)$$
3. **Return**: $b$ and $w$

In [ ]:
# Data
np.random.seed(42)
N = 100                                             # number of data
x = np.linspace(0, 10, N)
true_b = 1.0
true_w = 2.5
y = true_b + true_w * x + np.random.randn(N)        # model

# Hyperparameters
lr = 0.01
n_iters = 5                                      # stopping condition
record_gap = 1
record = True

# Initialization
b = 0
w = 0

# History
b_history = np.array([b])
w_history = np.array([w])

# GD
for i in range(n_iters):
    y_pred = w * x + b
    error = y_pred - y

    der_b = 2 * np.mean(error)
    der_w = 2 * np.mean(error * x)

    b = b - lr * der_b
    w = w - lr * der_w
    loss = np.mean(error ** 2)

    b_history = np.append(b_history, b)
    w_history = np.append(w_history, w)

    if record:
        if i % record_gap == 0:
            print(f"{'Step ' + str(i):<25}: b = {b:.4f}, w = {w:.4f}, target function = {loss:.4f}")

print(f"{'Final':<25}: b = {b:.4f}, w = {w:.4f}, target function = {loss:.4f}")

# Closed-form solution
X = np.vstack([x, np.ones_like(x)]).T
w_soln, b_soln = np.linalg.inv(X.T @ X) @ X.T @ y
print(f"{'Closed-form solution':<25}: b = {b_soln:.4f}, w = {w_soln:.4f}")

# Plot
fig, ax = plt.subplots(figsize = (8,6))
ax.scatter(x, y, s=10,label="Data", color="black")
if record:
    for i in range(n_iters//record_gap):
        ax.plot(x, w_history[i*record_gap] * x + b_history[i*record_gap], label=f"{i*record_gap}th iteration")
else:
    ax.plot(x, w * x + b, label=f"Final")

ax.plot(x, w_soln * x + b_soln, label=f"Solution")
ax.legend()
ax.set_title("Simple Linear Regression using Gradient Descent")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

#### Gradient
For $L$, We define
$$ \nabla L = \begin{bmatrix}
\frac{\partial L}{\partial b} \\
\\
\frac{\partial L}{\partial w}
\end{bmatrix} $$
With matrix form, the updating rule of GD can be written as
$$ \begin{bmatrix}
b \\
w
\end{bmatrix} \leftarrow \begin{bmatrix}
b \\
w
\end{bmatrix} - \eta \nabla L(b, w) $$

One can naturally extend simple linear regression to multiple linear regression with notion of gradient:
$$ y = b + w_1 x_1 + w_2 x_2 + \dots + w_d x_d + \epsilon $$